# Mapping Water Bodies in Leipzig

## Importing our modules

Python works with modules, `ee` is the Earth Engine API module, while `eemont` extends this API for a cleaner code-writing and `geemap` allows to plot Earth Engine results interactively.

In [3]:
import ee # Google Earth Engine API
import eemont # Extended GEE API
import geemap # Interactive Mapping with GEE
from geemap import colormaps as cm # Color Palettes

## Authentication

We need to authenticate to use Google Earth Engine, if this is your first time, please sign up [here](https://earthengine.google.com/).

In [5]:
# ee.Authenticate()
ee.Initialize() # Handles log-in for GEE

## Vector Data

Since we need the polygon for Leipzig, we have to load it from somewhere. Google Earth Engine stores vector data in three ways:

- ee.Geometry -> Imagine a shapefile with no attributes, just the geometry.
- ee.Feature -> A geometry with attributes.
- ee.FeatureCollection -> A full shapefile: Bunch of geometries and their attributes.

GEE has the Administrative Units of FAO GAUL stored in its catalog, so, let's use them!

The path to this dataset is "FAO/GAUL/2015/level2" and it's public!

In [6]:
FAO_GAUL = ee.FeatureCollection("FAO/GAUL/2015/level2") # Level 2 of Administrative Units (Vector)
LEIPZIG = FAO_GAUL.filter("ADM2_NAME == 'Leipzig'") # Filter to get the Adm. Region of Leipzig

To plot the Leipzig polygon, let's use `geemap`:

In [9]:
Map = geemap.Map() # Initialize an empty interactive Map
Map.addLayer(LEIPZIG,{},"Leipzig") # Add the Leipzig polygon to the map
Map.centerObject(LEIPZIG,8) # Center the map on Leipzig
Map # Plot the map

Map(center=[51.33712271360983, 12.682928568324028], controls=(WidgetControl(options=['position', 'transparent_…

## Processing Sentinel-2 Data

Now, we need the images of Sentinel-2 to extract our water bodies. Google Earth Engine stores raster data in two ways:

- ee.Image -> A single image with its properties. Images can have multiple bands!
- ee.ImageCollection -> A bunch of images. They don't need to be from the same sensor!

GEE has the complete dataset of Sentinel-2 images for the whole world! And they keep updating it as soon as new images arrive!

The path to this dataset is "COPERNICUS/S2_SR" (if we want the ARD)!

In [10]:
S2 = (ee.ImageCollection("COPERNICUS/S2_SR") # Sentinel-2 Surface Reflectance Collection (Raster)
      .filterBounds(LEIPZIG) # Filter the collection (just images that intersect Saxony)
      .filterDate("2021-06-01","2021-07-01") # Filter the collection -again- (just images in June 2021)      
      .median() # Composite (create a single median image from all images in the collection)
      .clip(LEIPZIG)) # Clip the composite to the Saxony boundaries

Now that we have filtered the collection, cerated a median composite and clipped it to the Leipzig boundaries, we want to plot it!

But We need to create some visualization parameters before that!

In [11]:
# For raster objects we need to define some visualization parameters
visualization = {
    "min": 0, # The minimum value for stretching the histogram
    "max": 3000, # The maximum value for stretching the histogram
    "bands": ["B4","B3","B2"] # The band combination (3-bands or single-band, 3-bands in this case)
}

Now, we can plot it!

In [12]:
Map = geemap.Map() # Initialize an empty map
Map.addLayer(S2,visualization,"S2 RGB") # Add the image to the map using the visualization parameters
Map.centerObject(LEIPZIG,8) # Center the map on Leipzig
Map # Plot the map

Map(center=[51.33712271360983, 12.682928568324028], controls=(WidgetControl(options=['position', 'transparent_…

## Ups! We forgot something important...

We have to mask the clouds and cloud-shadows! And it would be nice to scale the reflectance values to [0,1].

In [13]:
S2 = (ee.ImageCollection("COPERNICUS/S2_SR") # Sentinel-2 Surface Refelctance Collection
      .filterBounds(LEIPZIG) # Filter the collection (just images that intersect Saxony)
      .filterDate("2021-06-01","2021-07-01") # Filter the collection -again- (just images in June 2021)
      # ------- NEW LINE BELOW -------- #
      .preprocess() # Mask clouds, cloud-shadows and scale and offset each image
      # ------- END OF NEW LINE ------- #
      .median() # Composite (create a single median image from all images in the collection)
      .clip(LEIPZIG)) # Clip the composite to the Saxony boundaries

Let's define again our visualization parameters!

In [14]:
visualization = {
    "min": 0, # The minimum value
    "max": 0.3, # The maximum value (note the scale is different)
    "bands": ["B4","B3","B2"] # The band combination
}

And let's plot the result!

In [15]:
Map = geemap.Map() # Initialize an empty map
Map.addLayer(S2,visualization,"S2 RGB") # Add the image to the map using the visualization parameters
Map.centerObject(LEIPZIG,8) # Center the map on Leipzig
Map # Plot the map

Map(center=[51.33712271360983, 12.682928568324028], controls=(WidgetControl(options=['position', 'transparent_…

## And the Water?

There is a simple way to know if a pixel is water or not: Using band math!

- "Band math?"
- "Yes! We compute new images by combining single bands!"

There are some combinations useful for detecting and monitoring vegetation, urban zones, and water! The Normalized Difference Water Index (NDWI) is computed as a normalized difference between the GREEN and the NIR bands (`NDWI = (G-N)/(G+N)`) of an image. This index moves between -1 and 1. The more close to 1, the better indicator of a presence of water.

Let's compute the NDWI to extract water!

In [16]:
S2 = (ee.ImageCollection("COPERNICUS/S2_SR") # Sentinel-2 Surface Refelctance Collection
      .filterBounds(LEIPZIG) # Filter the collection (just images that intersect Saxony)
      .filterDate("2021-06-01","2021-07-01") # Filter the collection -again- (just images in June 2021)
      .preprocess() # Mask clouds, cloud-shadows and scale and offset each image
      # ------- NEW LINE BELOW -------- #
      .spectralIndices("NDWI") # Compute the Normalized Difference Water Index
      # ------- END OF NEW LINE ------- #
      .median() # Composite (create a single median image from all images in the collection)
      .clip(LEIPZIG)) # Clip the composite to the Saxony boundaries

Let's define our visualization parameters for the NDWI.

In [17]:
visualization = {
    "min": 0, # Minimum value
    "max": 1, # Maximum value
    "palette": cm.palettes.ndwi, # Color palette
    "bands": "NDWI" # The band combination (in this case a single band)
}

And let's plot it!

In [18]:
Map = geemap.Map() # Initialize an empty map
Map.addLayer(S2,visualization,"NDWI") # Add the NDWI to the map
Map.centerObject(LEIPZIG,8) # Center the map on Leipzig
Map # Plot the map

Map(center=[51.33712271360983, 12.682928568324028], controls=(WidgetControl(options=['position', 'transparent_…

## Finally, the Water Bodies!

If we set a threshold on the NDWI, we can decide what is water and what is not water. the most common threshold is zero. NDWI values greater than zero are considered water pixels.

In [19]:
WATER = S2["NDWI"] > 0 # All pixels where NDWI is greater than zero are considered water pixels

This logical operation gives us an image with two values: 0 and 1. 0 is not water. 1 is water. Let's define our visualization parameters to see the result!

In [20]:
visualization = {
    "min": 0, # Minimum value
    "max": 1, # Maximum value
    "palette": cm.palettes.ndwi # Color palette
}

Let's plot it!

In [21]:
Map = geemap.Map() # Initialize an empty map
Map.addLayer(WATER,visualization,"Water Bodies") # Add water bodies to the map
Map.centerObject(LEIPZIG,8) # Center map on Leipzig
Map # Plot the map

Map(center=[51.33712271360983, 12.682928568324028], controls=(WidgetControl(options=['position', 'transparent_…

## I want to make a map in QGIS with this result!

Well, there are two options to do this. The first one is "downloading" the image. GEE allows you to export the result to your Drive account, that you can download later.

In [22]:
task = ee.batch.Export.image.toDrive(image = WATER,
                                     description = "Water_bodies_Leipzig",
                                     region = LEIPZIG.geometry(),
                                     scale = 10,
                                     maxPixels = 1e12)

task.start()

Here you can check the status of this exporting task:

In [25]:
task.status()

{'state': 'READY',
 'description': 'Water_bodies_Leipzig',
 'creation_timestamp_ms': 1625743777932,
 'update_timestamp_ms': 1625743777932,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'CMOQM7MHB3HU6AOPXMQNJQVN',
 'name': 'projects/earthengine-legacy/operations/CMOQM7MHB3HU6AOPXMQNJQVN'}

The second option is to WORK WITH GOOGLE EARTH ENGINE INSIDE QGIS!

- "Wait, is that possible?"

Yes, it is! You just need to install the [GEE Plugin for QGIS](https://gee-community.github.io/qgis-earthengine-plugin/) and [install eemont in QGIS](https://eemont.readthedocs.io/en/latest/guide/eemontR.html). There is no need to install `geemap` in QGIS since all visualization are done using the QGIS visualization schema!